<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch

torch.cuda.is_available()

start = time.time()

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.sigmoid(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
train_start = time.time()

# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

end = time.time()
print(f'Total: {end - start} sec')
print(f'Train: {end - train_start} sec')


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0625
Current benign train loss: 2.3492236137390137

Current batch: 100
Current benign train accuracy: 0.0703125
Current benign train loss: 5.182206153869629

Current batch: 200
Current benign train accuracy: 0.2109375
Current benign train loss: 2.4182920455932617

Current batch: 300
Current benign train accuracy: 0.25
Current benign train loss: 2.135697841644287

Current batch: 400
Current benign train accuracy: 0.265625
Current benign train loss: 2.0192222595214844

Current batch: 500
Current benign train accuracy: 0.234375
Current benign train loss: 2.3121511936187744

Current batch: 600
Current benign train accuracy: 0.3046875
Current benign train loss: 1.8807868957519531

Current batch: 700
Current benign train accuracy: 0.2734375
Current benign train loss: 1.9448450803756714

Current batch: 800
Current benign train accuracy: 0.3046875
Current benign train loss: 2.068021297454834

Current batch: 900
Current beni


Current batch: 800
Current benign train accuracy: 0.6328125
Current benign train loss: 1.0776687860488892

Current batch: 900
Current benign train accuracy: 0.6328125
Current benign train loss: 1.055666446685791

Current batch: 1000
Current benign train accuracy: 0.6796875
Current benign train loss: 0.9878197908401489

Current batch: 1100
Current benign train accuracy: 0.6328125
Current benign train loss: 1.028078317642212

Total benign train accuarcy: 62.88666666666666
Total benign train loss: 1240.400471150875

[ Test epoch: 5 ]

Test accuarcy: 14.35
Test average loss: 0.05345925192832947
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.6796875
Current benign train loss: 0.8633548021316528

Current batch: 100
Current benign train accuracy: 0.6328125
Current benign train loss: 0.9834551215171814

Current batch: 200
Current benign train accuracy: 0.6328125
Current benign train loss: 0.9031366109848022

Current batch: 300
Current benign train accuracy


Current batch: 300
Current benign train accuracy: 0.7265625
Current benign train loss: 0.7459074854850769

Current batch: 400
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8223289847373962

Current batch: 500
Current benign train accuracy: 0.6875
Current benign train loss: 0.8949071764945984

Current batch: 600
Current benign train accuracy: 0.6875
Current benign train loss: 0.9175135493278503

Current batch: 700
Current benign train accuracy: 0.59375
Current benign train loss: 1.1044602394104004

Current batch: 800
Current benign train accuracy: 0.671875
Current benign train loss: 0.8806690573692322

Current batch: 900
Current benign train accuracy: 0.703125
Current benign train loss: 0.8084313869476318

Current batch: 1000
Current benign train accuracy: 0.6875
Current benign train loss: 0.8476454019546509

Current batch: 1100
Current benign train accuracy: 0.78125
Current benign train loss: 0.6488659381866455

Total benign train accuarcy: 70.08666666666667
To


Current batch: 1100
Current benign train accuracy: 0.75
Current benign train loss: 0.6454094052314758

Total benign train accuarcy: 72.922
Total benign train loss: 919.9750204384327

[ Test epoch: 16 ]

Test accuarcy: 28.21
Test average loss: 0.040673703384399414
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.75
Current benign train loss: 0.6447681188583374

Current batch: 100
Current benign train accuracy: 0.703125
Current benign train loss: 0.7831945419311523

Current batch: 200
Current benign train accuracy: 0.75
Current benign train loss: 0.755021333694458

Current batch: 300
Current benign train accuracy: 0.6953125
Current benign train loss: 0.844983696937561

Current batch: 400
Current benign train accuracy: 0.7734375
Current benign train loss: 0.7092356085777283

Current batch: 500
Current benign train accuracy: 0.75
Current benign train loss: 0.8999550938606262

Current batch: 600
Current benign train accuracy: 0.7890625
Current benign tra


Current batch: 500
Current benign train accuracy: 0.734375
Current benign train loss: 0.7474083304405212

Current batch: 600
Current benign train accuracy: 0.75
Current benign train loss: 0.6832873225212097

Current batch: 700
Current benign train accuracy: 0.7109375
Current benign train loss: 0.7911984324455261

Current batch: 800
Current benign train accuracy: 0.6953125
Current benign train loss: 0.9424688220024109

Current batch: 900
Current benign train accuracy: 0.765625
Current benign train loss: 0.7231923341751099

Current batch: 1000
Current benign train accuracy: 0.75
Current benign train loss: 0.7941787242889404

Current batch: 1100
Current benign train accuracy: 0.7578125
Current benign train loss: 0.6793902516365051

Total benign train accuarcy: 74.696
Total benign train loss: 861.4316051900387

[ Test epoch: 22 ]

Test accuarcy: 18.6
Test average loss: 0.07708372974395752
Model Saved!

[ Train epoch: 23 ]

Current batch: 0
Current benign train accuracy: 0.7109375
Current 

...


Current batch: 700
Current benign train accuracy: 0.9140625
Current benign train loss: 0.3104761838912964

Current batch: 800
Current benign train accuracy: 0.90625
Current benign train loss: 0.2970814108848572

Current batch: 900
Current benign train accuracy: 0.875
Current benign train loss: 0.30973073840141296

Current batch: 1000
Current benign train accuracy: 0.84375
Current benign train loss: 0.33507364988327026

Current batch: 1100
Current benign train accuracy: 0.9375
Current benign train loss: 0.16320152580738068

Total benign train accuarcy: 90.23333333333333
Total benign train loss: 331.4735952243209

[ Test epoch: 72 ]

Test accuarcy: 67.21
Test average loss: 0.016084913313388825
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.3432580232620239

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2262493371963501

Current batch: 200
Current benign train accuracy: 0.882


Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09785285592079163

Current batch: 200
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16523396968841553

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.17821766436100006

Current batch: 400
Current benign train accuracy: 0.9375
Current benign train loss: 0.15179432928562164

Current batch: 500
Current benign train accuracy: 0.9140625
Current benign train loss: 0.22307632863521576

Current batch: 600
Current benign train accuracy: 0.9453125
Current benign train loss: 0.19101259112358093

Current batch: 700
Current benign train accuracy: 0.96875
Current benign train loss: 0.15814609825611115

Current batch: 800
Current benign train accuracy: 0.9375
Current benign train loss: 0.1750936061143875

Current batch: 900
Current benign train accuracy: 0.9609375
Current benign train loss: 0.17713457345962524

Current batch: 1000
Current benign trai


Current batch: 900
Current benign train accuracy: 0.9453125
Current benign train loss: 0.2013290375471115

Current batch: 1000
Current benign train accuracy: 0.9453125
Current benign train loss: 0.14097967743873596

Current batch: 1100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08746717125177383

Total benign train accuarcy: 94.85133333333333
Total benign train loss: 188.19606233760715

[ Test epoch: 83 ]

Test accuarcy: 88.48
Test average loss: 0.0036284620761871337
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.266198068857193

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1481194645166397

Current batch: 200
Current benign train accuracy: 0.9609375
Current benign train loss: 0.12676769495010376

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.14558100700378418

Current batch: 400
Current benign train acc


Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08256704360246658

Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.16751912236213684

Current batch: 400
Current benign train accuracy: 0.9375
Current benign train loss: 0.19353646039962769

Current batch: 500
Current benign train accuracy: 0.984375
Current benign train loss: 0.09576933830976486

Current batch: 600
Current benign train accuracy: 0.9375
Current benign train loss: 0.12786449491977692

Current batch: 700
Current benign train accuracy: 0.921875
Current benign train loss: 0.1981852501630783

Current batch: 800
Current benign train accuracy: 0.9140625
Current benign train loss: 0.21768981218338013

Current batch: 900
Current benign train accuracy: 0.8984375
Current benign train loss: 0.21589511632919312

Current batch: 1000
Current benign train accuracy: 0.9296875
Current benign train loss: 0.15795555710792542

Current batch: 1100
Current benign 


Current batch: 900
Current benign train accuracy: 0.9453125
Current benign train loss: 0.13661479949951172

Current batch: 1000
Current benign train accuracy: 0.9609375
Current benign train loss: 0.12163706868886948

Current batch: 1100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.1501588225364685

Total benign train accuarcy: 95.26666666666667
Total benign train loss: 171.4158484339714

[ Test epoch: 94 ]

Test accuarcy: 87.21
Test average loss: 0.004300091178715229
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.15556037425994873

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.09789086878299713

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.15623025596141815

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.11326158046722412

Current batch: 400
Current benign train accura